In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os
import torchvision.models as models
from torchvision import transforms
import pandas as pd
from torch.utils.data.sampler import  WeightedRandomSampler

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE=10

In [3]:
csv = pd.read_csv('./images/dataset/0_index.csv',usecols=[0,2,4])#第4列是氨纶，用不上，我懒得改了
csv_numpy = csv.to_numpy()

In [4]:
ss,DD,sD,Ds = [],[],[],[]
for i in range(len(csv_numpy)):
    if csv_numpy[i][1] == 's/s':
        ss.append(csv_numpy[i])
    elif csv_numpy[i][1] == 'D/D':
        DD.append(csv_numpy[i])
    elif csv_numpy[i][1] == 'D/s':
        Ds.append(csv_numpy[i])
    elif csv_numpy[i][1] == 's/D':
        sD.append(csv_numpy[i])
print(len(ss))
print(len(DD))
print(len(Ds))
print(len(sD))

225
188
41
31


In [5]:
train_ratio = 0.7
test_ratio = 0.3
train_set, test_set = [],[]
#这四个for以后想想能不能优化一下
for i in range(len(ss)):
    temp = ['./images/dataset/' + ss[i][0], 0]
    if i<int(train_ratio*len(ss)):
        train_set.append(temp)
    else:
        test_set.append(temp)
for i in range(len(DD)):
    temp = ['./images/dataset/' + DD[i][0], 1]
    if i<int(train_ratio*len(DD)):
        train_set.append(temp)
    else:
        test_set.append(temp)
'''for i in range(len(Ds)):
    temp = ['./images/dataset/' + Ds[i][0], 2]
    if i<int(train_ratio*len(Ds)):
        train_set.append(temp)
    else:
        test_set.append(temp)
for i in range(len(sD)):
    temp = ['./images/dataset/' + sD[i][0], 3]
    if i<int(train_ratio*len(sD)):
        train_set.append(temp)
    else:
        test_set.append(temp)'''
print(len(train_set),len(test_set))

288 125


In [6]:
class MyDataset(Dataset):
    def __init__(self, imgs, transform = None, target_transform = None):       
        self.imgs = imgs * 3 #将数据暴力复制3份，后面会做transforms
        self.transform = transform
        self.target_transform = target_transform
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB') 
        if self.transform is not None:
            img = self.transform(img) 
        return img, label
    def __len__(self):
        return len(self.imgs)

In [7]:
#图像的初始化操作
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

#数据集加载方式设置
train_data = MyDataset(train_set, transform=train_transforms)
test_data = MyDataset(test_set, transform=test_transforms)

#由于样本不均衡，因此做重采样
'''weights = []
for data,target in train_set:
    if target in (2,3):
        weights.append(4) #长短和短长的权重翻n倍
    else:
        weights.append(1)
sampler = WeightedRandomSampler(weights, num_samples=1450, replacement=True)#1020是随便取了一个小于(924+(24+6)*3)的,batchsize的倍数'''

#然后就是调用DataLoader和刚刚创建的数据集，来创建dataloader，这里提一句，loader的长度是有多少个batch，所以和batch_size有关
#train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,num_workers=0,sampler=sampler)
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,num_workers=0,shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
print('num_of_trainloader:', len(train_loader))
print('num_of_testloader:', len(test_loader))

num_of_trainloader: 87
num_of_testloader: 38


In [13]:
t = 0
d = 0
for (data,target) in train_loader:
    t=target
    d=data
    break
d.shape

torch.Size([10, 3, 224, 224])

In [8]:
def train(model,device, train_loader, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        x,y= data
        x=x.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        y_hat= model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print ('Train Epoch: {}\t Loss: {:.6f}'.format(epoch,loss.item()))
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i,data in enumerate(test_loader):          
            x,y= data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            test_loss += criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_data),
        100. * correct / len(test_data)))

In [10]:
#resnet18 = models.resnet18(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
#vgg16 = models.vgg16(pretrained=True)

model_conv = models.alexnet(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.classifier[6].in_features
model_conv.classifier[6] = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params':model_conv.classifier[6].parameters()}
], lr=0.001)

In [11]:
for epoch in range(1, 10):
    train(model=model_conv, device=DEVICE, train_loader=train_loader,epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader)

Train Epoch: 1	 Loss: 0.040668

Test set: Average loss: 0.0430, Accuracy: 324/375 (86%)

Train Epoch: 2	 Loss: 0.079861

Test set: Average loss: 0.0375, Accuracy: 318/375 (85%)

Train Epoch: 3	 Loss: 0.062890

Test set: Average loss: 0.0349, Accuracy: 339/375 (90%)

Train Epoch: 4	 Loss: 0.269283

Test set: Average loss: 0.0372, Accuracy: 330/375 (88%)

Train Epoch: 5	 Loss: 0.498316

Test set: Average loss: 0.0480, Accuracy: 318/375 (85%)

Train Epoch: 6	 Loss: 0.000041

Test set: Average loss: 0.0513, Accuracy: 327/375 (87%)

Train Epoch: 7	 Loss: 0.088198

Test set: Average loss: 0.0419, Accuracy: 330/375 (88%)

Train Epoch: 8	 Loss: 0.007337

Test set: Average loss: 0.0724, Accuracy: 315/375 (84%)

Train Epoch: 9	 Loss: 0.133690

Test set: Average loss: 0.0500, Accuracy: 321/375 (86%)



In [15]:
#resnet18 = models.resnet18(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
#vgg16 = models.vgg16(pretrained=True)

model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params':model_conv.fc.parameters()}
], lr=0.001)

RuntimeError: CUDA error: device-side assert triggered

In [45]:
for epoch in range(1, 10):
    train(model=model_conv, device=DEVICE, train_loader=train_loader,epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader)

Train Epoch: 1	 Loss: 0.501058

Test set: Average loss: 0.0405, Accuracy: 387/477 (81%)

Train Epoch: 2	 Loss: 0.257031

Test set: Average loss: 0.0322, Accuracy: 405/477 (85%)

Train Epoch: 3	 Loss: 0.571294

Test set: Average loss: 0.0207, Accuracy: 447/477 (94%)

Train Epoch: 4	 Loss: 0.645221

Test set: Average loss: 0.0222, Accuracy: 435/477 (91%)

Train Epoch: 5	 Loss: 0.595838

Test set: Average loss: 0.0205, Accuracy: 444/477 (93%)

Train Epoch: 6	 Loss: 0.256903

Test set: Average loss: 0.0224, Accuracy: 435/477 (91%)

Train Epoch: 7	 Loss: 0.422598

Test set: Average loss: 0.0212, Accuracy: 432/477 (91%)

Train Epoch: 8	 Loss: 0.217016

Test set: Average loss: 0.0198, Accuracy: 435/477 (91%)

Train Epoch: 9	 Loss: 0.264859

Test set: Average loss: 0.0233, Accuracy: 420/477 (88%)



In [10]:
#resnet18 = models.resnet18(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
#vgg16 = models.vgg16(pretrained=True)

model_conv = models.vgg16(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
#model_conv.classifier[0] = nn.Linear(25088, 4096)
num_second_last = model_conv.classifier[3].in_features
model_conv.classifier[3] = nn.Linear(num_second_last, num_second_last)
num_last = model_conv.classifier[6].in_features
model_conv.classifier[6] = nn.Linear(num_last, 4)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam([
    {'params':model_conv.classifier[3].parameters()},
    {'params':model_conv.classifier[6].parameters()}
], lr=0.001)

In [17]:
model_conv

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
for epoch in range(1, 10):
    train(model=model_conv, device=DEVICE, train_loader=train_loader,epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader)

RuntimeError: cuda runtime error (710) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1570910687650/work/aten/src/THC/generic/THCTensorMath.cu:26